# Homework 1 Notebook

You can follow the `homework.md` file to get the answer and the code used to calculate them are included in this notebook

In [3]:
import requests
import datetime
import pandas as pd
import os

from evidently import ColumnMapping
from evidently.report import Report
from evidently.metrics import ColumnDriftMetric, DatasetDriftMetric, DatasetMissingValuesMetric, DatasetCorrelationsMetric, ColumnQuantileMetric

from joblib import load, dump
from tqdm import tqdm

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

We download data from March 2024

In [6]:

files = [('green_tripdata_2024-03.parquet', './data')]

# Create the data directory
os.makedirs('./data', exist_ok=True)

print("Download files:")
for file, path in files:
    url=f"https://d37ci6vzurychx.cloudfront.net/trip-data/{file}"
    resp=requests.get(url, stream=True)
    save_path=f"{path}/{file}"
    with open(save_path, "wb") as handle:
        for data in tqdm(resp.iter_content(),
                        desc=f"{file}",
                        postfix=f"save to {save_path}",
                        total=int(resp.headers["Content-Length"])):
            handle.write(data)

Download files:


green_tripdata_2024-03.parquet: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1372372/1372372 [00:09<00:00, 152109.32it/s, save to ./data/green_tripdata_2024-03.parquet]


In [7]:
march_data = pd.read_parquet('data/green_tripdata_2024-03.parquet')

In [8]:
march_data.describe()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
count,57457.000000,57457,57457,55360.000000,57457.000000,57457.000000,55360.000000,57457.000000,57457.000000,57457.000000,57457.000000,57457.000000,57457.000000,0.0,57457.000000,57457.000000,55360.000000,55353.000000,55360.000000
mean,1.877334,2024-03-16 04:02:52.405399,2024-03-16 04:21:00.076039,1.179986,95.524688,138.629149,1.309538,13.522828,17.313474,0.904472,0.577410,2.386255,0.192537,NaN,0.979378,22.904832,1.321062,1.038047,0.737730
min,1.000000,2008-12-31 23:02:24,2008-12-31 23:02:30,1.000000,1.000000,1.000000,0.000000,0.000000,-295.080000,-2.500000,-0.500000,-1.560000,0.000000,NaN,-1.000000,-296.080000,1.000000,1.000000,-2.750000
25%,2.000000,2024-03-08 13:53:56,2024-03-08 14:13:49,1.000000,74.000000,74.000000,1.000000,1.100000,9.300000,0.000000,0.500000,0.000000,0.000000,NaN,1.000000,13.440000,1.000000,1.000000,0.000000
50%,2.000000,2024-03-15 22:49:01,2024-03-15 23:09:52,1.000000,75.000000,138.000000,1.000000,1.790000,13.500000,0.000000,0.500000,2.000000,0.000000,NaN,1.000000,18.500000,1.000000,1.000000,0.000000
75%,2.000000,2024-03-23 20:11:25,2024-03-23 20:34:48,1.000000,97.000000,220.000000,1.000000,3.100000,19.800000,1.000000,0.500000,3.610000,0.000000,NaN,1.000000,27.050000,2.000000,1.000000,2.750000
max,2.000000,2024-04-01 00:01:45,2024-04-01 16:11:00,99.000000,265.000000,265.000000,9.000000,125112.200000,841.600000,10.000000,4.250000,150.000000,26.760000,NaN,1.000000,856.980000,5.000000,2.000000,2.750000
std,0.328056,NaN,NaN,1.356719,57.285088,76.295346,0.967749,770.416255,14.958249,1.382446,0.366916,3.159273,1.184551,NaN,0.154253,17.013735,0.497858,0.191311,1.218039


In [9]:
march_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57457 entries, 0 to 57456
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   VendorID               57457 non-null  int32         
 1   lpep_pickup_datetime   57457 non-null  datetime64[us]
 2   lpep_dropoff_datetime  57457 non-null  datetime64[us]
 3   store_and_fwd_flag     55360 non-null  object        
 4   RatecodeID             55360 non-null  float64       
 5   PULocationID           57457 non-null  int32         
 6   DOLocationID           57457 non-null  int32         
 7   passenger_count        55360 non-null  float64       
 8   trip_distance          57457 non-null  float64       
 9   fare_amount            57457 non-null  float64       
 10  extra                  57457 non-null  float64       
 11  mta_tax                57457 non-null  float64       
 12  tip_amount             57457 non-null  float64       
 13  t

In [10]:
march_data.shape

(57457, 20)

In [7]:
with open('models/lin_reg.bin', 'rb') as f_out:
    model = load(f_out)

In [8]:
# create target
march_data["duration_min"] = march_data.lpep_dropoff_datetime - march_data.lpep_pickup_datetime
march_data.duration_min = march_data.duration_min.apply(lambda td : float(td.total_seconds())/60)

In [11]:
# data labeling
target = "duration_min"
num_features = ["passenger_count", "trip_distance", "fare_amount", "total_amount"]
cat_features = ["PULocationID", "DOLocationID"]

In [10]:
################ NO ################
march_preds = model.predict(march_data[num_features + cat_features])
march_data['prediction'] = march_preds

ValueError: Input X contains NaN.
LinearRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [12]:
column_mapping = ColumnMapping(
    target=None,
    prediction='prediction',
    numerical_features=num_features,
    categorical_features=cat_features
)

# Evidently Dashboard

In [13]:
from evidently.metric_preset import DataDriftPreset, DataQualityPreset

from evidently.ui.workspace import Workspace
from evidently.ui.dashboards import DashboardPanelCounter, DashboardPanelPlot, CounterAgg, PanelValue, PlotType, ReportFilter
from evidently.renderers.html_widgets import WidgetSize

Create a workspace and a project

In [14]:
ws = Workspace("workspace")

In [15]:
project = ws.create_project("NYC Taxi March 2024 Evaluation Reports")
project.description = "My project Fare Amount measure"
project.save()

Project(id=UUID('f8b1a3fb-5890-41ac-9dbd-58a47e7e9f36'), name='NYC Taxi March 2024 Evaluation Reports', description='My project Fare Amount measure', dashboard=DashboardConfig(name='NYC Taxi March 2024 Evaluation Reports', panels=[], tabs=[], tab_id_to_panel_ids={}), team_id=None, date_from=None, date_to=None, created_at=datetime.datetime(2024, 6, 23, 9, 1, 33, 639690))

We run a report for every day between start and end date

In [16]:
def add_report(metrics, column_mapping, curr_data, ref_data, start_date, end_date, project_id, ws):
    # Generate tuples with start and end date for every day in 
    dates = [ (start_date + datetime.timedelta(n), start_date + datetime.timedelta(n+1))  
             for n in range(int((end_date - start_date).days))]
    for start,end in dates:
        # DEfine the report
        regular_report = Report(
            metrics=metrics,
            timestamp=start
        )
        #curr_data_filtered= curr_data.lpep_pickup_datetime.between(pd.Timestamp(start),pd.Timestamp(end), inclusive="left")
        curr_data_filtered= curr_data.lpep_pickup_datetime.between(start.strftime('%Y-%m-%d'),end.strftime('%Y-%m-%d'), inclusive="left")
        if curr_data_filtered.any():
            # run the report
            regular_report.run(reference_data=ref_data,
                            current_data=curr_data.loc[curr_data.lpep_pickup_datetime.between(start.strftime('%Y-%m-%d'),end.strftime('%Y-%m-%d'), 
                                                                                             inclusive="left")],
                            column_mapping=column_mapping)
    
            # Add the report to the workspace
            ws.add_report(project_id, regular_report)
            #print("Report executed")
        else:
           regular_report=None 

    return regular_report
        

In [17]:
march_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57457 entries, 0 to 57456
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   VendorID               57457 non-null  int32         
 1   lpep_pickup_datetime   57457 non-null  datetime64[us]
 2   lpep_dropoff_datetime  57457 non-null  datetime64[us]
 3   store_and_fwd_flag     55360 non-null  object        
 4   RatecodeID             55360 non-null  float64       
 5   PULocationID           57457 non-null  int32         
 6   DOLocationID           57457 non-null  int32         
 7   passenger_count        55360 non-null  float64       
 8   trip_distance          57457 non-null  float64       
 9   fare_amount            57457 non-null  float64       
 10  extra                  57457 non-null  float64       
 11  mta_tax                57457 non-null  float64       
 12  tip_amount             57457 non-null  float64       
 13  t

In [ ]:
metrics=[
        DataQualityPreset(),
        ColumnQuantileMetric(column_name='fare_amount', quantile=0.5),
]
curr_data=march_data
ref_data=None
start_date = datetime.datetime(2024, 3, 1)
end_date   = datetime.datetime(2024, 3, 31)
project_id= project.id

result_test = add_report(metrics, column_mapping, curr_data, ref_data, start_date, end_date, project_id, ws)

In [19]:
project.save()

Project(id=UUID('f8b1a3fb-5890-41ac-9dbd-58a47e7e9f36'), name='NYC Taxi March 2024 Evaluation Reports', description='My project Fare Amount measure', dashboard=DashboardConfig(name='NYC Taxi March 2024 Evaluation Reports', panels=[], tabs=[], tab_id_to_panel_ids={}), team_id=None, date_from=None, date_to=None, created_at=datetime.datetime(2024, 6, 23, 9, 1, 33, 639690))

In [ ]:
result = result_test.as_dict()
result

### Define the dashboard

In [21]:
#configure the dashboard
project.dashboard.add_panel(
    DashboardPanelCounter(
        filter=ReportFilter(metadata_values={}, tag_values=[]),
        agg=CounterAgg.NONE,
        title="NYC taxi March 2024 Dashboard"
    )
)

project.dashboard.add_panel(
    DashboardPanelPlot(
        filter=ReportFilter(metadata_values={}, tag_values=[]),
        title="Fare Amount Quantile Count",
        values=[
            PanelValue(
                metric_id="ColumnQuantileMetric",
                field_path="current.value",
                legend="count"
            ),
        ],
        plot_type=PlotType.BAR,
        size=WidgetSize.HALF,
    ),
)

In [22]:
project.save()

Project(id=UUID('f8b1a3fb-5890-41ac-9dbd-58a47e7e9f36'), name='NYC Taxi March 2024 Evaluation Reports', description='My project Fare Amount measure', dashboard=DashboardConfig(name='NYC Taxi March 2024 Evaluation Reports', panels=[DashboardPanelCounter(type='evidently.ui.dashboards.reports.DashboardPanelCounter', id=UUID('6bbe91d0-5b96-4c64-8314-9ed57d56aeda'), title='NYC taxi March 2024 Dashboard', filter=ReportFilter(metadata_values={}, tag_values=[], include_test_suites=False), size=<WidgetSize.FULL: 2>, agg=<CounterAgg.NONE: 'none'>, value=None, text=None), DashboardPanelPlot(type='evidently.ui.dashboards.reports.DashboardPanelPlot', id=UUID('3b91ab70-4873-49d1-afc4-4c00ef381526'), title='Fare Amount Quantile Count', filter=ReportFilter(metadata_values={}, tag_values=[], include_test_suites=False), size=<WidgetSize.HALF: 1>, values=[PanelValue(field_path='current.value', metric_id='ColumnQuantileMetric', metric_fingerprint=None, metric_args={}, legend='count')], plot_type=<PlotType

run the Evidently ui to check the dashboard and reports

`evidently ui`

### Adding more panels to the dashboard

In [23]:

project.dashboard.add_panel(
    DashboardPanelPlot(
        filter=ReportFilter(metadata_values={}, tag_values=[]),
        title="Inference Count",
        values=[
            PanelValue(
                metric_id="DatasetSummaryMetric",
                field_path="current.number_of_rows",
                legend="count"
            ),
        ],
        plot_type=PlotType.BAR,
        size=WidgetSize.HALF,
    ),
)

project.dashboard.add_panel(
    DashboardPanelPlot(
        filter=ReportFilter(metadata_values={}, tag_values=[]),
        title="Number of Missing Values",
        values=[
            PanelValue(
                metric_id="DatasetSummaryMetric",
                field_path="current.number_of_missing_values",
                legend="count"
            ),
        ],
        plot_type=PlotType.LINE,
        size=WidgetSize.HALF,
    ),
)


In [24]:
project.save()

Project(id=UUID('f8b1a3fb-5890-41ac-9dbd-58a47e7e9f36'), name='NYC Taxi March 2024 Evaluation Reports', description='My project Fare Amount measure', dashboard=DashboardConfig(name='NYC Taxi March 2024 Evaluation Reports', panels=[DashboardPanelCounter(type='evidently.ui.dashboards.reports.DashboardPanelCounter', id=UUID('6bbe91d0-5b96-4c64-8314-9ed57d56aeda'), title='NYC taxi March 2024 Dashboard', filter=ReportFilter(metadata_values={}, tag_values=[], include_test_suites=False), size=<WidgetSize.FULL: 2>, agg=<CounterAgg.NONE: 'none'>, value=None, text=None), DashboardPanelPlot(type='evidently.ui.dashboards.reports.DashboardPanelPlot', id=UUID('3b91ab70-4873-49d1-afc4-4c00ef381526'), title='Fare Amount Quantile Count', filter=ReportFilter(metadata_values={}, tag_values=[], include_test_suites=False), size=<WidgetSize.HALF: 1>, values=[PanelValue(field_path='current.value', metric_id='ColumnQuantileMetric', metric_fingerprint=None, metric_args={}, legend='count')], plot_type=<PlotType